In [7]:
from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Input, UpSampling2D
from tensorflow.keras import backend as k
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Model


In [8]:
(x_train, _), (x_test, _) = mnist.load_data()

x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

x_train = np.reshape(x_train, (len(x_train), 28, 28, 1))
x_test = np.reshape(x_test, (len(x_test), 28, 28, 1))

print(x_train.shape, x_test.shape)

(60000, 28, 28, 1) (10000, 28, 28, 1)


In [9]:
input_img = Input(shape=(28, 28, 1))

#encoding
x = Conv2D(16, (3, 3), activation='relu', padding='same')(input_img)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
encoded = MaxPooling2D((2, 2), padding='same')(x)

#decoding
x = Conv2D(8, (3, 3), activation='relu', padding='same')(encoded)
x = UpSampling2D((2, 2))(x)
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
x = Conv2D(16, (3, 3), activation='relu')(x)
x = UpSampling2D((2, 2))(x)
decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)

autoencoder = Model(input_img, decoded)
autoencoder.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
autoencoder.fit(x_train, x_train, epochs=100, batch_size=64, shuffle = True, validation_data=(x_test, x_test))

Epoch 1/100
938/938 [==============================] - 43s 44ms/step - loss: 0.1720 - accuracy: 0.7990 - val_loss: 0.1265 - val_accuracy: 0.8060
Epoch 2/100
938/938 [==============================] - 33s 35ms/step - loss: 0.1212 - accuracy: 0.8083 - val_loss: 0.1150 - val_accuracy: 0.8091
Epoch 3/100
938/938 [==============================] - 33s 35ms/step - loss: 0.1128 - accuracy: 0.8100 - val_loss: 0.1090 - val_accuracy: 0.8092
Epoch 4/100
938/938 [==============================] - 33s 35ms/step - loss: 0.1081 - accuracy: 0.8109 - val_loss: 0.1053 - val_accuracy: 0.8106
Epoch 5/100
938/938 [==============================] - 32s 34ms/step - loss: 0.1049 - accuracy: 0.8114 - val_loss: 0.1022 - val_accuracy: 0.8111
Epoch 6/100
938/938 [==============================] - 33s 35ms/step - loss: 0.1024 - accuracy: 0.8118 - val_loss: 0.1002 - val_accuracy: 0.8109
Epoch 7/100
938/938 [==============================] - 32s 34ms/step - loss: 0.1006 - accuracy: 0.8121 - val_loss: 0.0987 - val_ac

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



938/938 [==============================] - 39s 42ms/step - loss: 0.0883 - accuracy: 0.8139 - val_loss: 0.0871 - val_accuracy: 0.8128
Epoch 49/100
938/938 [==============================] - 33s 35ms/step - loss: 0.0883 - accuracy: 0.8139 - val_loss: 0.0873 - val_accuracy: 0.8131
Epoch 50/100
291/938 [========>.....................] - ETA: 21s - loss: 0.0882 - accuracy: 0.8138

In [ ]:
loss, accuracy = autoencoder.evaluate(x_test, x_test, verbose=0)
print(f'loss: {(loss*100):.2f} || Accuracy: {(accuracy*100):.2f}')

In [ ]:
num_images = 10
np.random.seed(42)
random_test_images = np.random.randit(x_test.shape[0], size=num_images)

decoded_imgs = autoencoder.predict(x_test)

plt.figure(figsize=(18, 4))

for i, image_idx in enumerate(random_test_images):
    ax = plt.subplot(3, num_images, i+1)
    plt.imshow(x_test[image_idx].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    ax = plt.subplot(3, num_images, 2*num_images+i+1)
    plt.imshow(decoded_imgs[image_idx].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()